In [ ]:
"""

Date: June 3, 2022

Author: Karine Choquet

This script will produce datasets for Figures 1 and S1 of the splicing order manuscript



"""

In [1]:
import numpy as np
import pandas as pd
import pysam
import seaborn as sns
sns.set_style("white")
sns.set_style("ticks")

import matplotlib.pyplot as plt
import re
%matplotlib inline
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rc('xtick', labelsize=14) 
plt.rc('ytick', labelsize=14)

import matplotlib.ticker as ticker
from matplotlib.ticker import ScalarFormatter

import math

import pybedtools
from pybedtools import BedTool

from collections import Counter

import random


import scipy
from scipy import stats
from scipy.stats import ttest_ind, ttest_ind_from_stats
from scipy.special import stdtr
from scipy.stats import chi2_contingency

import itertools

from statsmodels.stats.multitest import multipletests

from more_itertools import consecutive_groups

from tqdm import tqdm

In [2]:
# Load intron features and gene_names df
hg38_intron_df = pd.read_table("/path/to/annotation_files/hg38_all_intron_features.txt")

hg38_intron_df = hg38_intron_df[['gene','intron_total','strand']].drop_duplicates().reset_index(drop=True)



## Splicing order and poly(A) tail length

In [3]:
# Import data (splicing status per read)
chr1_multi_introns_df = pd.read_table("/path/to/K562_chr_rep1_hg38_multi_introns_isoforms_df.RefSeq.txt")
chr2_multi_introns_df = pd.read_table("/path/to/K562_chr_rep2_hg38_multi_introns_isoforms_df.RefSeq.txt")

chr_multi_introns_df = pd.concat([chr1_multi_introns_df,chr2_multi_introns_df]).reset_index(drop=True)
chr_multi_introns_df = chr_multi_introns_df.merge(hg38_intron_df, on=['gene','strand'])

In [4]:
# Import splicing order data
path_df_nodup = pd.read_table('/path/to/TableS3_K562_splicing_order_paths.txt')
path_df_nodup_top = path_df_nodup[path_df_nodup['rank']==1].reset_index(drop=True)

In [5]:
# Define intron groups
regions_df = path_df_nodup_top[['gene','gene_name','analyzed_introns','n_analyzed_introns']].drop_duplicates().reset_index(drop=True)
regions_df = regions_df.merge(hg38_intron_df, on='gene')

In [7]:
# Change intron number from genomic position to position in transcript
def reset_intron_numbers(introns_of_interest, n_introns_gene, strand):

    introns_of_interest_list_tmp = introns_of_interest.split("_")
    if strand == "-":
        introns_of_interest_list = [str(n_introns_gene - int(i)) for i in introns_of_interest_list_tmp]
        introns_of_interest_fix = "_".join(introns_of_interest_list)
    elif strand == "+":
        introns_of_interest_list = [str(int(i)+1) for i in introns_of_interest_list_tmp]
        introns_of_interest_fix = "_".join(introns_of_interest_list)

    return introns_of_interest_fix

In [8]:
# Change intron number
chr_multi_introns_df['new_intron_numbers'] = chr_multi_introns_df.apply(lambda row: reset_intron_numbers(row.intron_numbers, row.intron_total, row.strand),axis=1)

In [9]:
# Get splicing status only for introns within intron group
def code_intron_status(df, n_introns, analyzed_introns_list):
    
    results_list = []
    
    for row in range(len(df)):
        readname = df.loc[row]['read']
        strand = df.loc[row]['strand']
        intron_numbers = df.loc[row]['new_intron_numbers']
        splice_status_introns = df.loc[row]['splice_status']
    
        intron_numbers_list = [int(a) for a in intron_numbers.split("_")]
        splice_status_list_introns = splice_status_introns.split("_")
        
        introns_of_interest_pos = [i for i, x in enumerate(intron_numbers_list) if x in analyzed_introns_list]
        splice_status_list_sub = [splice_status_list_introns[a] for a in introns_of_interest_pos]
        
        new_splice_status_list = [readname] + splice_status_list_sub
            
        results_list.append(new_splice_status_list)
    results_df = pd.DataFrame(results_list)
    results_df.columns = ['readname'] + analyzed_introns_list
            
    return(results_df)
            
    
    

In [10]:
# Apply function above to each intron group
mat_list = []

for i in tqdm(range(len(regions_df))):
    gene = regions_df.loc[i]['gene']
    analyzed_introns = regions_df.loc[i]['analyzed_introns']
    n_analyzed_introns = regions_df.loc[i]['n_analyzed_introns']
    intron_total = regions_df.loc[i]['intron_total']
    strand = regions_df.loc[i]['strand']
    
    print(gene, analyzed_introns)
    
    analyzed_introns_list = [int(a) for a in analyzed_introns.split("_")]
    
    # in case the list contains non-consecutive numbers, fill it in
    new_analyzed_introns_list = list(range(min(analyzed_introns_list), max(analyzed_introns_list)+1))
    if strand == "-":
        new_analyzed_introns_list.reverse()
    new_analyzed_introns_list_str = [str(a) for a in new_analyzed_introns_list]
    new_analyzed_introns = "_".join(new_analyzed_introns_list_str)
    
    column_list = ['readname'] + analyzed_introns_list

    df = chr_multi_introns_df[(chr_multi_introns_df['gene']==gene) & 
                              ((chr_multi_introns_df['new_intron_numbers'].str.contains(new_analyzed_introns)) |
                               (chr_multi_introns_df['new_intron_numbers'].str.contains(analyzed_introns)))].reset_index(drop=True)
    mat = code_intron_status(df, intron_total+1, analyzed_introns_list)
    mat2 = mat.copy()[column_list]
    
    if n_analyzed_introns == 3:
        mat2.columns = ['readname','int1','int2','int3']
        mat2['splice_status'] = mat2['int1'] + '_' + mat2['int2'] + '_' + mat2['int3']
    elif n_analyzed_introns == 4:
        mat2.columns = ['readname','int1','int2','int3','int4']
        mat2['splice_status'] = mat2['int1'] + '_' + mat2['int2'] + '_' + mat2['int3'] + '_' + mat2['int4']
       
    mat_final = mat2.copy()[['readname','splice_status']]
    mat_final['gene'] = gene
    mat_final['analyzed_introns'] = analyzed_introns
    mat_final['n_analyzed_introns'] = n_analyzed_introns
    mat_list.append(mat_final)
    
mat_all = pd.concat(mat_list)

  0%|          | 0/669 [00:00<?, ?it/s]

NM_000018.3 10_11_12_13


  0%|          | 1/669 [00:01<21:02,  1.89s/it]

NM_000018.3 11_12_13_14


  0%|          | 2/669 [00:03<18:31,  1.67s/it]

...

100%|█████████▉| 667/669 [16:43<00:02,  1.44s/it]

NM_199337.2 1_2_3


100%|█████████▉| 668/669 [16:44<00:01,  1.44s/it]

NM_213720.2 3_2_1


100%|██████████| 669/669 [16:46<00:00,  1.50s/it]


In [11]:
mat_all.to_csv("/path/to/K562_splicing_status_per_read_intron_groups.txt", sep="\t", header=True, index=False)


## Analysis of EXOSC10 KD

In [2]:
# Function to get the number of reads mapping to each intermediate isoform

def get_pattern_counts(multi_introns_df, total_introns, introns_of_interest, n_introns_gene, strand):
    
    t=0

    # Make a dictionary with the patterns and the counts and another with the number of introns spliced and the counts
    pattern_dict = {}
    results_list = []
    spliced_counts_dict = {}
    
    introns_of_interest_list = introns_of_interest.split("_")
    
    # Initiate pattern_dict
    pattern_dict = {}
    #for n in range(len(introns_of_interest_list)+1):
    #    pattern_dict[n] = {}
        
    # Filter for patterns that are present with a certain threshold
    multi_introns_df = multi_introns_df[(multi_introns_df['count']>t) & (~multi_introns_df['splice_status'].str.contains("UND"))].reset_index(drop=True)
    
    # Iterate to get counts for isoforms
    for row in range(len(multi_introns_df)):
        gene_name = multi_introns_df.loc[row]['gene_name']
        splice_status_temp = multi_introns_df.loc[row]['splice_status']
        intron_numbers_list_tmp = multi_introns_df.loc[row]['intron_numbers'].split("_")
        if strand == "-":
            intron_numbers_list = [str(n_introns_gene - int(i)) for i in intron_numbers_list_tmp]
            intron_numbers = "_".join(intron_numbers_list)
        elif strand == "+":
            intron_numbers_list = [str(int(i)+1) for i in intron_numbers_list_tmp]
            intron_numbers = "_".join(intron_numbers_list)
            
            
        if introns_of_interest in intron_numbers:
            introns_of_interest_pos = [i for i, x in enumerate(intron_numbers_list) if x in introns_of_interest_list]
            splice_status_list_temp1 = splice_status_temp.split("_")
            splice_status_list_temp = [splice_status_list_temp1[a] for a in introns_of_interest_pos]
            splice_status_list = ["SKP" if "SKP" in a else a for a in splice_status_list_temp]
            splice_status = "_".join(splice_status_list)
            pattern_count = multi_introns_df.loc[row]['count']
            skipped_count = Counter(splice_status_list)['SKP']
            spliced_count = Counter(splice_status_list)['YES']
            unspliced_count = Counter(splice_status_list)['NO']
            if skipped_count == 0: # no skipped introns
                level = spliced_count
                if skipped_count < total_introns:
                    if splice_status not in pattern_dict.keys():
                        pattern_dict[splice_status] = pattern_count
                    elif splice_status in pattern_dict.keys():
                        pattern_dict[splice_status] += pattern_count
                        
    pattern_df = pd.DataFrame.from_dict(pattern_dict, orient='index').reset_index()
    
    if len(pattern_df) > 1:
        pattern_df.columns = ['pattern','count']
            
        return(pattern_df)
    
    
# Apply the function above to all genes of interest in a given sample
def get_read_count_path_per_gene(multi_introns_df, cand_gene_df):
    
    results_list = []
    
    for i in range(len(cand_gene_df)):
        gene = cand_gene_df.loc[i]['gene']
        gene_name = cand_gene_df.loc[i]['gene_name']
        analyzed_introns = cand_gene_df.loc[i]['analyzed_introns']
        n_analyzed_introns = len(analyzed_introns.split("_"))
        
        for sample_name in sample_list:
        
            df = multi_introns_df[(multi_introns_df['gene']==gene) & (multi_introns_df['sample_name']==sample_name)].reset_index(drop=True)
            total_introns = len(analyzed_introns.split("_"))
            n_introns_gene = int(hg38_intron_df.loc[hg38_intron_df['gene']==gene]['intron_total'].drop_duplicates())
            strand = "".join(hg38_intron_df.loc[hg38_intron_df['gene']==gene]['strand'].drop_duplicates().tolist())
        
            results_df = get_pattern_counts(df, total_introns, analyzed_introns, n_introns_gene, strand)
        
            if results_df is None:
                continue
            else:
                results_df['gene_name'] = gene_name
                results_df['gene'] = gene
                results_df['analyzed_introns'] = analyzed_introns
                results_df['n_analyzed_introns'] = n_analyzed_introns
                results_df['sample_name'] = sample_name
            
                results_list.append(results_df)
            
    final_df = pd.concat(results_list)
        
    return(final_df)
    
    

In [4]:
# Load intron features and gene_names df
hg38_intron_df = pd.read_table("/path/to/annotation_files/hg38_all_intron_features.txt")
gene_names_df = pd.read_table("/path/to/annotation_files/hg38_UCSC_refGene_names.txt")
gene_names_df.columns = ['gene_name','gene_id']

# Merge gene and intron annotations
hg38_intron_df['gene_id'] = hg38_intron_df['gene'].str.split("\\.").str[0]
gene_names_df = gene_names_df.merge(hg38_intron_df, on='gene_id')[['gene_name','gene','intron_total']].drop_duplicates().reset_index(drop=True)
hg38_intron_coord = hg38_intron_df.copy()[['chrom','start','end','gene','intron_pos']]




In [7]:
# Load results from cluster from identifying splicing order paths in both WT K562 replicates
path_df_tmp = pd.read_table('/path/to/K562_chromatin_splicing_paths_non_consecutive_introns.2mergedReps.RefSeq.max4introns.txt')
path_df = path_df_tmp[['sample_name','gene','gene_name','analyzed_introns','n_analyzed_introns','full_path','full_path_score','rank']].drop_duplicates().sort_values(by=['gene_name','analyzed_introns','rank']).reset_index(drop=True)

# Remove duplicates intron groups because they belong to different transcripts
intron_groups = path_df[['gene','gene_name','analyzed_introns','n_analyzed_introns']].drop_duplicates().reset_index(drop=True)

intron_groups_3 = intron_groups[intron_groups['n_analyzed_introns']==3].reset_index(drop=True)
intron_groups_3['int1'] = intron_groups_3['analyzed_introns'].str.split("_").str[0].astype(int)
intron_groups_3['int2'] = intron_groups_3['analyzed_introns'].str.split("_").str[1].astype(int)
intron_groups_3['int3'] = intron_groups_3['analyzed_introns'].str.split("_").str[2].astype(int)

intron_groups_3 = intron_groups_3.merge(hg38_intron_coord, left_on=['gene','int1'], right_on=['gene','intron_pos']).rename(columns={'chrom':'chrom_1', 'start':'start_1', 'end':'end_1'})
intron_groups_3 = intron_groups_3.merge(hg38_intron_coord, left_on=['gene','int2'], right_on=['gene','intron_pos']).rename(columns={'chrom':'chrom_2', 'start':'start_2', 'end':'end_2'})
intron_groups_3 = intron_groups_3.merge(hg38_intron_coord, left_on=['gene','int3'], right_on=['gene','intron_pos']).rename(columns={'chrom':'chrom_3', 'start':'start_3', 'end':'end_3'})

intron_groups_3 = intron_groups_3.sort_values(by=['gene','int1','int2','int3']).drop_duplicates(subset=['chrom_1','start_1','end_1','chrom_2','start_2','end_2','chrom_3','start_3','end_3']).reset_index(drop=True)

intron_groups_4 = intron_groups[intron_groups['n_analyzed_introns']==4].reset_index(drop=True)
intron_groups_4['int1'] = intron_groups_4['analyzed_introns'].str.split("_").str[0].astype(int)
intron_groups_4['int2'] = intron_groups_4['analyzed_introns'].str.split("_").str[1].astype(int)
intron_groups_4['int3'] = intron_groups_4['analyzed_introns'].str.split("_").str[2].astype(int)
intron_groups_4['int4'] = intron_groups_4['analyzed_introns'].str.split("_").str[3].astype(int)

intron_groups_4 = intron_groups_4.merge(hg38_intron_coord, left_on=['gene','int1'], right_on=['gene','intron_pos']).rename(columns={'chrom':'chrom_1', 'start':'start_1', 'end':'end_1'})
intron_groups_4 = intron_groups_4.merge(hg38_intron_coord, left_on=['gene','int2'], right_on=['gene','intron_pos']).rename(columns={'chrom':'chrom_2', 'start':'start_2', 'end':'end_2'})
intron_groups_4 = intron_groups_4.merge(hg38_intron_coord, left_on=['gene','int3'], right_on=['gene','intron_pos']).rename(columns={'chrom':'chrom_3', 'start':'start_3', 'end':'end_3'})
intron_groups_4 = intron_groups_4.merge(hg38_intron_coord, left_on=['gene','int4'], right_on=['gene','intron_pos']).rename(columns={'chrom':'chrom_4', 'start':'start_4', 'end':'end_4'})

intron_groups_4 = intron_groups_4.sort_values(by=['gene','int1','int2','int3','int4']).drop_duplicates(subset=['chrom_1','start_1','end_1','chrom_2','start_2','end_2','chrom_3','start_3','end_3','chrom_4','start_4','end_4']).reset_index(drop=True)

fields = ['gene','gene_name','analyzed_introns','n_analyzed_introns']
intron_groups_nodup = pd.concat([intron_groups_3[fields],intron_groups_4[fields]]).sort_values(by=['gene_name','analyzed_introns']).reset_index(drop=True)

# Merge back with paths
path_df_tmp_nodup = path_df_tmp.merge(intron_groups_nodup, on=['gene','gene_name','analyzed_introns','n_analyzed_introns'])
path_df_nodup = path_df_tmp_nodup[['sample_name','gene','gene_name','analyzed_introns','n_analyzed_introns','full_path','full_path_score','rank']].drop_duplicates().sort_values(by=['gene_name','analyzed_introns','rank']).reset_index(drop=True)

# Reformat the table to have replicates side by side
path_df_rep1 = path_df_nodup[path_df_nodup['sample_name']=='chr_rep1'].reset_index(drop=True)
path_df_rep2 = path_df_nodup[path_df_nodup['sample_name']=='chr_rep2'].reset_index(drop=True)

path_df_piv = path_df_rep1.merge(path_df_rep2, on=['gene_name','gene','analyzed_introns','n_analyzed_introns','full_path'])

# Retrieve the paths that are reproducible (same rank in both samples)
path_df_reprod = path_df_piv[(path_df_piv['rank_x']==path_df_piv['rank_y'])].reset_index(drop=True)


In [8]:
# Get intermediate isoform counts from scramble and EXOSC10 KD
scr1_multi_intron_counts = pd.read_table("/path/to/K562_scrambled_KD_nucRNA_hg38_multi_introns_isoforms_counts.withSKP.RefSeq.txt")
exo1_multi_intron_counts = pd.read_table("/path/to/K562_EXOSC10_KD_nucRNA_hg38_multi_introns_isoforms_counts.withSKP.RefSeq.txt")
scr2_multi_intron_counts = pd.read_table("/path/to/K562_scramble_KD_nucRNA_rep2_hg38_multi_introns_isoforms_counts.withSKP.RefSeq.txt")
exo2_multi_intron_counts = pd.read_table("/path/to/K562_EXOSC10_KD_nucRNA_rep2_hg38_multi_introns_isoforms_counts.withSKP.RefSeq.txt")

# Add sample name
scr1_multi_intron_counts['sample_name'] = "scrambled_KD_rep1"
exo1_multi_intron_counts['sample_name'] = "EXOSC10_KD_rep1"
scr2_multi_intron_counts['sample_name'] = "scrambled_KD_rep2"
exo2_multi_intron_counts['sample_name'] = "EXOSC10_KD_rep2"

# Merge with gene names
scr1_multi_intron_counts = scr1_multi_intron_counts.merge(gene_names_df, on='gene')
exo1_multi_intron_counts = exo1_multi_intron_counts.merge(gene_names_df, on='gene')
scr2_multi_intron_counts = scr2_multi_intron_counts.merge(gene_names_df, on='gene')
exo2_multi_intron_counts = exo2_multi_intron_counts.merge(gene_names_df, on='gene')

# Concatenate all samples
multi_intron_counts = pd.concat([scr1_multi_intron_counts,exo1_multi_intron_counts,
                                scr2_multi_intron_counts,exo2_multi_intron_counts]).reset_index(drop=True)

sample_list = multi_intron_counts['sample_name'].drop_duplicates().tolist()

In [9]:
# Get a list of intron groups to analyze
cand_gene_df = path_df_reprod[['gene','gene_name','analyzed_introns']].drop_duplicates().reset_index(drop=True)

# Get read counts
test_df = get_read_count_path_per_gene(multi_intron_counts, cand_gene_df)

# Normalize counts per gene region
counts_per_region = pd.DataFrame(test_df.groupby(['sample_name','gene_name','gene','analyzed_introns','n_analyzed_introns'])['count'].sum()).reset_index().rename(columns={'count':'total_count'})

test_df2 = test_df.merge(counts_per_region, on=['sample_name','gene_name','gene','analyzed_introns','n_analyzed_introns'])
test_df2['norm_count'] = test_df2['count'] / test_df2['total_count']

# Add the splicing level
test_df2['level'] = test_df2['pattern'].str.count(r'YES')

# Calculate log10 of counts
test_df2['log10_count'] = np.log10(test_df2['norm_count'])

# Write to file for plotting in R
test_df2.to_csv('/path/to/EXOSC10_KD_read_counts_for_intron_groups_in_paths_with_Reps.txt', sep="\t", header=True, index=False)


In [10]:
# Pivot table
test_df_piv = test_df2.pivot_table(index=['gene_name','gene','analyzed_introns','pattern','level'], columns='sample_name', values='count').fillna(0).reset_index()

In [11]:
# Function to compare the levels of intermediate isoforms per splicing level

def compare_isoforms_by_level(gene, analyzed_introns, pattern, scr, exo):
    
    # Retrieve all the other patterns for that intron group at that level
    pattern_count_scr = int(test_df_piv_sub[(test_df_piv_sub['gene']==gene) & (test_df_piv_sub['analyzed_introns']==analyzed_introns) &
                                   (test_df_piv_sub['pattern']==pattern)][scr])
    pattern_count_exo = int(test_df_piv_sub[(test_df_piv_sub['gene']==gene) & (test_df_piv_sub['analyzed_introns']==analyzed_introns) &
                                   (test_df_piv_sub['pattern']==pattern)][exo])
    
    total_count_scr = int(test_df_piv_sub[(test_df_piv_sub['gene']==gene) & (test_df_piv_sub['analyzed_introns']==analyzed_introns) &
                                   (test_df_piv_sub['pattern']!=pattern)][scr].sum())
    total_count_exo = int(test_df_piv_sub[(test_df_piv_sub['gene']==gene) & (test_df_piv_sub['analyzed_introns']==analyzed_introns) &
                                   (test_df_piv_sub['pattern']!=pattern)][exo].sum())
    
    OR, pvalue = scipy.stats.fisher_exact([[pattern_count_exo,total_count_exo],[pattern_count_scr,total_count_scr]])
    
    return(OR, pvalue)
    

In [12]:
# Filter for a minimum coverage in at least one condition, and only analyze intermediate isoforms
test_df_piv_sub = test_df_piv[(((test_df_piv['scrambled_KD_rep1']>10) & (test_df_piv['scrambled_KD_rep2']>10)) |
                         ((test_df_piv['EXOSC10_KD_rep1']>10) & (test_df_piv['EXOSC10_KD_rep2']>10))) &
                             ((test_df_piv['level']>0))].reset_index(drop=True)

# Compare conditions using function defined above
test_df_piv_sub['OR_rep1'] = test_df_piv_sub.apply(lambda row: compare_isoforms_by_level(row.gene, row.analyzed_introns, row.pattern, "scrambled_KD_rep1", "EXOSC10_KD_rep1")[0],axis=1)
test_df_piv_sub['OR_rep2'] = test_df_piv_sub.apply(lambda row: compare_isoforms_by_level(row.gene, row.analyzed_introns, row.pattern, "scrambled_KD_rep2", "EXOSC10_KD_rep2")[0],axis=1)
test_df_piv_sub['pvalue_rep1'] = test_df_piv_sub.apply(lambda row: compare_isoforms_by_level(row.gene, row.analyzed_introns, row.pattern, "scrambled_KD_rep1", "EXOSC10_KD_rep1")[1],axis=1)
test_df_piv_sub['pvalue_rep2'] = test_df_piv_sub.apply(lambda row: compare_isoforms_by_level(row.gene, row.analyzed_introns, row.pattern, "scrambled_KD_rep2", "EXOSC10_KD_rep2")[1],axis=1)

# Correct for multiple testing
test_df_piv_sub['FDR_rep1'] = multipletests(test_df_piv_sub['pvalue_rep1'], alpha=0.05, method='fdr_bh')[1]
test_df_piv_sub['FDR_rep2'] = multipletests(test_df_piv_sub['pvalue_rep2'], alpha=0.05, method='fdr_bh')[1]

# Add a category for statistical significance
test_df_piv_sub['sig_category'] = 'non-significant'
test_df_piv_sub.loc[(test_df_piv_sub['FDR_rep1']<0.1) & (test_df_piv_sub['FDR_rep2']<0.1) &
                    ((test_df_piv_sub['OR_rep1']>1) & (test_df_piv_sub['OR_rep2']>1)) &
                    (test_df_piv_sub['pattern'].str.contains("NO")), 'sig_category'] = 'significant'



# Retrieve intron groups that show significant differences
test_df_piv_sig = test_df_piv_sub[(test_df_piv_sub['sig_category']=='significant') & (test_df_piv_sub['pattern'].str.contains("YES")) & (test_df_piv_sub['pattern'].str.contains("NO")) & (test_df_piv_sub['OR_rep1']>1)].reset_index(drop=True)

In [13]:
# Write to file for for Supplemental Table
test_df_piv_sub.to_csv('/path/to/TableS4_EXOSC10_KD_pattern_comparison_fishers_exact_test_with_reps.txt', sep="\t", header=True, index=False)
